In [1]:
import pandas as pd
import numpy as np

## 코스피 레이블링하기

In [90]:
df = pd.read_csv('코스피_라벨피쳐_전처리.csv',index_col = 0)
df = df.iloc[:,:6]
df.isna().sum()

회사명                                     0
거래소코드                                   0
회계년도                                    0
이자보상배율(이자비용)(IFRS연결)                  224
영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)    224
자본(*)(IFRS연결)(천원)                     224
dtype: int64

##  코스피 라벨 피쳐 3개가 모두 결측치인거 제거

In [91]:
ind = df[df.iloc[:,3:].isna().all(axis = 1)].index
df = df.drop(index = ind).reset_index(drop=True)
df

,회사명,거래소코드,회계년도,이자보상배율(이자비용)(IFRS연결),영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원),자본(*)(IFRS연결)(천원)
0,(주)HJ매그놀리아용평호텔앤리조트,70960,2013/12,1.45,36775171.0,298067032.0
1,(주)HJ매그놀리아용평호텔앤리조트,70960,2014/12,1.07,40502835.0,301490946.0
2,(주)HJ매그놀리아용평호텔앤리조트,70960,2015/12,1.84,36832912.0,312817856.0
3,(주)HJ매그놀리아용평호텔앤리조트,70960,2016/12,2.62,-18994098.0,396442087.0
4,(주)HJ매그놀리아용평호텔앤리조트,70960,2017/12,3.00,17809537.0,408181173.0
...,...,...,...,...,...,...
9400,흥아해운(주),3280,2018/12,-1.16,-33794586.0,73148175.0
9401,흥아해운(주),3280,2019/12,-0.55,-12048291.0,30919383.0
9402,흥아해운(주),3280,2020/12,-0.38,-3958237.0,-26283865.0
9403,흥아해운(주),3280,2021/12,-1.06,-2635682.0,102939788.0


### 코스피 동일 회사 중복 제거 및 해외 법인 회사 제거

In [92]:
temp = df[df['회사명'] == '에스케이(주)']
ind = temp[df['거래소코드'] == 3600].index
df = df.drop(index = ind)

ind= df[df["거래소코드"] > 900000].index.tolist()
df = df.drop(index = ind).reset_index(drop= True)

C:\Users\geonh\AppData\Local\Temp\ipykernel_39696\767628473.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ind = temp[df['거래소코드'] == 3600].index


In [95]:
## 코스피 라벨 피쳐 정렬하기

In [96]:
df = df.sort_values(['거래소코드','회사명','회계년도'])
df = df.reset_index(drop=True)
df

,회사명,거래소코드,회계년도,이자보상배율(이자비용)(IFRS연결),영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원),자본(*)(IFRS연결)(천원)
0,동화약품(주),20,2010/12,6.25,11628787.0,217153837.0
1,동화약품(주),20,2011/12,14.58,46244032.0,229623874.0
2,동화약품(주),20,2012/12,9.47,25226760.0,226994670.0
3,동화약품(주),20,2013/12,5.37,8205110.0,227231127.0
4,동화약품(주),20,2014/12,33.65,13514668.0,229725782.0
...,...,...,...,...,...,...
9364,두산로보틱스(주),454910,2018/12,844531.69,-17473671.0,25482765.0
9365,두산로보틱스(주),454910,2019/12,-46.05,-18082748.0,23571013.0
9366,두산로보틱스(주),454910,2020/12,-37.51,-4027165.0,9139203.0
9367,두산로보틱스(주),454910,2021/12,-17.19,-7244230.0,15502080.0


## 이자보상배율 1미만일 경우(CONDITION1) 영업활동으로 인한 현금흐름,자본(CONDITION2)

In [97]:
def condition1(x):
    if x< 1:
        return -1
    else :
        return 0
        
def condition2_3(x):
    if x< 0:
        return -1
    else :
        return 0

### 함수 각 LABEL Feature에 mapping

In [98]:
df['이자보상배율'] = df['이자보상배율(이자비용)(IFRS연결)'].map(condition1)
df['현금흐름'] = df['영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)'].map(condition2_3)
df['자본'] = df['자본(*)(IFRS연결)(천원)'].map(condition2_3)
df['이자_부실'] = 0
df['현금흐름_부실'] = 0
df['자본_부실'] = 0

### 코스피 자본이 최근년도 음일경우 부실->1

In [99]:
df['자본_부실'] =  df['자본'].map( lambda x: 1 if x < 0 else 0)
df['부실'] = df['자본'].map( lambda x: 1 if x < 0 else 0)

### 코스피 이자보상배율 3개년도 1미만일시 부실 ->1

In [100]:
df2 = pd.DataFrame()
for i in df['회사명'].unique():
    temp = df[df['회사명'] == i]
    temp = temp.reset_index(drop = True)
    for j in range(len(temp) - 2):
        if temp.iloc[j:j+3,6].apply("sum") == -3:
            temp.iloc[j+2,-1] = 1
            temp.iloc[j+2,-4] = 1
    df2 = pd.concat([df2,temp],axis = 0)
df = df2

### 코스피 현금흐름 3개년도 음일경우 부실 ->1

In [101]:
df2 = pd.DataFrame()
for i in df['회사명'].unique():
    temp = df[df['회사명'] == i]
    temp = temp.reset_index(drop = True)
    for j in range(len(temp) - 2):
        if temp.iloc[j:j+3,7].apply("sum") == -3:
            temp.iloc[j+2,-1] = 1
            temp.iloc[j+2,-3] = 1
    df2 = pd.concat([df2,temp],axis = 0)
df = df2

In [102]:
df.to_csv('코스피_라벨링.csv')

## 코스닥 라벨링 하기

In [103]:
df = pd.read_csv('코스닥_라벨피쳐_전처리.csv',index_col = 0)
df = df.iloc[:,:6]
df.isna().sum()

회사명                                      0
거래소코드                                    0
회계년도                                     0
이자보상배율(이자비용)(IFRS연결)                  2357
영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)    2356
자본(*)(IFRS연결)(천원)                     2356
dtype: int64

## 3개년도 연속 결측치인것 제거

In [104]:
ind = np.where(df.iloc[:,3:].isna().all(axis=1))[0]
df = df.sort_values(['거래소코드','회사명','회계년도'])
df = df.drop(index = ind).reset_index(drop=True)
df.isna().sum()

회사명                                   0
거래소코드                                 0
회계년도                                  0
이자보상배율(이자비용)(IFRS연결)                  2
영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)    1
자본(*)(IFRS연결)(천원)                     1
dtype: int64

## 코스닥 스펙 합병 기업 제거

In [105]:
sp_enp =['(주)본느','(주)세화피앤씨','(주)아이엘사이언스','(주)알로이스','(주)에스에이티','(주)원바이오젠','(주)원익피앤이','(주)인산가','(주)자비스','(주)정다운','(주)카이노스메드','(주)티에스트릴리온','(주)포인트엔지니어링','(주)한국비엔씨','(주)휴럼','나무기술(주)','씨아이에스(주)','엠에프엠코리아(주)','원텍(주)','패션플랫폼(주)']
for i in sp_enp:
    ind = df[df['회사명'] == i].index
    df = df.drop(index = ind).reset_index(drop=True)
df.isna().sum()

회사명                                   0
거래소코드                                 0
회계년도                                  0
이자보상배율(이자비용)(IFRS연결)                  2
영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)    1
자본(*)(IFRS연결)(천원)                     1
dtype: int64

## 코스닥 해외 법인 회사 제거

In [106]:
ind= df[df["거래소코드"] > 900000].index.tolist()
df = df.drop(index = ind).reset_index(drop= True)
df.isna().sum()

회사명                                   0
거래소코드                                 0
회계년도                                  0
이자보상배율(이자비용)(IFRS연결)                  2
영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)    1
자본(*)(IFRS연결)(천원)                     1
dtype: int64

### 코스닥 결측치 확인

In [110]:
df[df['이자보상배율(이자비용)(IFRS연결)'].isna()]

,회사명,거래소코드,회계년도,이자보상배율(이자비용)(IFRS연결),영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원),자본(*)(IFRS연결)(천원)
13880,덕산네오룩스(주),213420,2014/12,NaN,0.0,102297139.0
14586,(주)슈프리마,236200,2015/12,NaN,0.0,77130099.0


In [111]:
df[df['영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)'].isna()]

,회사명,거래소코드,회계년도,이자보상배율(이자비용)(IFRS연결),영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원),자본(*)(IFRS연결)(천원)
16174,대명에너지(주),389260,2020/12,8432098.32,NaN,NaN


In [112]:
df[df['자본(*)(IFRS연결)(천원)'].isna()]

,회사명,거래소코드,회계년도,이자보상배율(이자비용)(IFRS연결),영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원),자본(*)(IFRS연결)(천원)
16174,대명에너지(주),389260,2020/12,8432098.32,NaN,NaN


In [113]:
# 결측치인것들은 모두 회사의 첫개년도 
#덕산네오룩스(주) == 2015년 상장 ->  2014년 데이터인 이자보상배율이 결측치 이지만 2015년도 이자보상배율은 1이상으로 부실기업과 관련 x
#(주)슈프리마 == 2016년 상장 -> 2015년 데이터중 이자보상배율이 결측치 이지만 2016년도 이자보상배율은 1이상으로 부실기업과 관련 x   
# 대명에너지 == 2022년 상장 -> 2020년 데이터중 현금흐름과 자본이 결측치 2021년도 현금흐름 0이상으로 부실과 관련 x , dart확인시 자본 양으로 인해 부실과 관련 x
## 3개 모두 부실 라벨 0으로 사용

### 함수 각 LABEL Feature에 mapping

In [114]:
df['이자보상배율'] = df['이자보상배율(이자비용)(IFRS연결)'].map(condition1)
df['현금흐름'] = df['영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)'].map(condition2_3)
df['자본'] = df['자본(*)(IFRS연결)(천원)'].map(condition2_3)
df['이자_부실'] = 0
df['현금흐름_부실'] = 0
df['자본_부실'] = 0

### 코스닥 자본이 최근년도 음일경우 부실->1

In [115]:
df['자본_부실'].map( lambda x: 1 if x < 0 else 0)
df['부실'] = df['자본'].map( lambda x: 1 if x < 0 else 0)

### 코스닥 이자보상배율 3개년도 1미만일시 부실 ->1

In [116]:
df2 = pd.DataFrame()
for i in df['회사명'].unique():
    temp = df[df['회사명'] == i]
    temp = temp.reset_index(drop = True)
    for j in range(len(temp) - 2):
        if temp.iloc[j:j+3,6].apply("sum") == -3:
            temp.iloc[j+2,-1] = 1
            temp.iloc[j+2,-4] = 1
    df2 = pd.concat([df2,temp],axis = 0)
df = df2

### 코스닥 현금흐름 3개년도 음일경우 부실 ->1

In [117]:
df2 = pd.DataFrame()
for i in df['회사명'].unique():
    temp = df[df['회사명'] == i]
    temp = temp.reset_index(drop = True)
    for j in range(len(temp) - 2):
        if temp.iloc[j:j+3,7].apply("sum") == -3:
            temp.iloc[j+2,-1] = 1
            temp.iloc[j+2,-3] = 1
    df2 = pd.concat([df2,temp],axis = 0)
df = df2

In [118]:
df.to_csv('코스닥_라벨링.csv')

In [130]:
#이자비용이나 부채가 감소하는 방향으로 가고있는 기업 선택